# Notebook para el despliegue del modelo en la nube de Azure

---

In [2]:
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Usamos una variable de entorno oculta para cargar el id de la suscripción, se trata de un archivo .env que debe estar en el mismo directorio que este script
my_secret = os.getenv("MY_SECRET")
tenant_id = os.getenv("TENANT_ID")

my_id = my_secret

from azureml.core.authentication import InteractiveLoginAuthentication
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)

from azureml.core import Workspace

ws = Workspace.create(name="workspace4",
                      subscription_id = my_id,
                      resource_group = "act-cloud-3",
                      location = "eastus2")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="linear_model.pkl",
                                  model_name=mname,
                                  workspace=ws)


Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/4130bd39-7c53-419c-b1e5-8758d6d63f21/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name workspacstorage9615a8962.
Deploying KeyVault with name workspackeyvault6c3ae54a.
Deploying AppInsights with name workspacinsightsfe37e627.
Deployed AppInsights with name workspacinsightsfe37e627. Took 1.93 seconds.
Deployed KeyVault with name workspackeyvault6c3ae54a. Took 18.21 seconds.
Deploying Workspace with name workspace4.
Deployed StorageAccount with name workspacstorage9615a8962. Took 24.68 seconds.
Deployed Workspace with name workspace4. Took 21.17 seconds.
Registering model model


In [3]:
################ score.py ###################


scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model
from datetime import date, timedelta

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)

def prepare_data(df:pd.DataFrame) -> pd.DataFrame:
    df = df[['FirstName', 'ModifiedDate']].sort_values(by='ModifiedDate')
    df['FirstNameLen'] = df['FirstName'].str.len()
    df["ModifiedDate"] = pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    df["ModifiedDate"] = df["ModifiedDate"].dt.days
    
    return df[['FirstNameLen', 'ModifiedDate']].reset_index(drop=True)

def int_to_date(day_int: int, start_date: date = date(2005, 7, 1)) -> date:
    
    result=start_date + timedelta(days=day_int)
    result = result.strftime("%Y-%m-%d")
    return result

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data = prepare_data(data)

    columns_model = ["FirstNameLen"]

    data_dummies = data[columns_model]

    result = model.predict(data_dummies).tolist()

    for i in range(len(result)):
      result[i] = int_to_date(result[i])
   

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-dates")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Usar los paquetes correctos


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

service = Model.deploy(workspace=ws,
                       name='service-2',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )